Adapted from : https://gist.github.com/DrDub/6efba6e522302e43d055

# example usage:
   f = FileBrowser(accept=".{filetype}")

   f.widget()

   <interact with widget, select a path>

# in a separate cell:
   f.path # returns the selected path

In [1]:
import os

class FileBrowser(object):
    def __init__(self, accept):
        self.path = os.getcwd()
        self._update_files()
        self.accept=accept
        self.layoutButtons = widgets.Layout(width='auto')
        self.buttonRun = widgets.Button(description='Run analysis', button_style='success', visible=False,
                                        layout=self.layoutButtons)
        self.output=widgets.Output(layout={'border': '1px solid black'})

    def _update_files(self):
        self.files = list()
        self.dirs = list()
        if(os.path.isdir(self.path)):
            for f in os.listdir(self.path):
                ff = os.path.join(self.path, f)
                if os.path.isdir(ff):
                    self.dirs.append(f)
                else:
                    self.files.append(f)

    def widget(self):
        box = widgets.VBox()
        self._update(box)
        return box

    def _update(self, box):
        def on_click(b):
            if b.description == '..':
                self.path = os.path.split(self.path)[0]
                with self.output:
                    clear_output()
            else:
                self.path = os.path.join(self.path, b.description)
            self._update_files()
            self._update(box)

        buttons = []
        button = widgets.Button(description='..', background_color='#d0d0ff', layout=self.layoutButtons)
        button.on_click(on_click)
        buttons.append(button)
        if(os.path.isdir(self.path)):
            for f in self.dirs:
                button = widgets.Button(description=f, background_color='#d0d0ff', layout=self.layoutButtons)
                button.on_click(on_click)
                buttons.append(button)
            for f in self.files:
                fileExtension = os.path.splitext(f)[1]
                if fileExtension == self.accept:
                    button = widgets.Button(description=f, button_style='success', layout=self.layoutButtons)
                    button.on_click(on_click)
                    buttons.append(button)
        if(os.path.isfile(self.path)):
            self.buttonRun.visible=True
            buttons.append(self.buttonRun)
            self.output.clear_output(wait=False)
            with self.output:
                print("Click to display an analytics dashboard for the chosen file")
            buttons.append(self.output)
        box.children = tuple([widgets.HTML("<h3>%s</h3>" % (self.path,))] + buttons)